In [1]:
import numpy as np
import pandas as pd

# 2 分类估计器

In [2]:
np.sqrt(3*3+4*4) #欧式距离
#欧式距离的定义是 两个向量之间的距离，就是两个向量的差的平方和的开方

5.0

In [3]:
# K近邻
"""
K-近邻预测用户签到位置
:return:None
"""
# 读取数据
data = pd.read_csv("../machine_learn/data/FBlocation/train.csv")
print(data.head(10))
print('-' * 50)
print(data.shape)#(29118021, 6)
print('-' * 50)
print(data.describe())
print('-' * 50)
# print(data.feature_names)
print('-' * 50)
# print(data.DESCR)

   row_id       x       y  accuracy    time    place_id
0       0  0.7941  9.0809        54  470702  8523065625
1       1  5.9567  4.7968        13  186555  1757726713
2       2  8.3078  7.0407        74  322648  1137537235
3       3  7.3665  2.5165        65  704587  6567393236
4       4  4.0961  1.1307        31  472130  7440663949
5       5  3.8099  1.9586        75  178065  6289802927
6       6  6.3336  4.3720        13  666829  9931249544
7       7  5.7409  6.7697        85  369002  5662813655
8       8  4.3114  6.9410         3  166384  8471780938
9       9  6.3414  0.0758        65  400060  1253803156
--------------------------------------------------
(29118021, 6)
--------------------------------------------------
             row_id             x             y      accuracy          time  \
count  2.911802e+07  2.911802e+07  2.911802e+07  2.911802e+07  2.911802e+07   
mean   1.455901e+07  4.999770e+00  5.001814e+00  8.284912e+01  4.170104e+05   
std    8.405649e+06  2.857601e+

In [4]:
# 处理数据
# 1、缩小数据,查询数据,为了减少计算时间
print('before')
print(data.shape)
print('-' * 50)
data = data.query("x > 1.0 &  x < 1.25 & y > 2.5 & y < 2.75")
print('after')
print(data.shape)
print('-' * 50)

before
(29118021, 6)
--------------------------------------------------
after
(17710, 6)
--------------------------------------------------


In [5]:
#2. 处理时间的数据
# 原time--->pd中的时间格式--->字典格式
# 处理原data中的time，变为pd中的时间格式
time_value = pd.to_datetime(data['time'], unit='s')
print(time_value)
print('-' * 50)
# 把日期格式转换成 字典格式，把年，月，日，时，分，秒转换为字典格式，
time_value = pd.DatetimeIndex(time_value)
print(time_value)
print('-' * 50)

600        1970-01-01 18:09:40
957        1970-01-10 02:11:10
4345       1970-01-05 15:08:02
4735       1970-01-06 23:03:03
5580       1970-01-09 11:26:50
                   ...        
29100203   1970-01-01 10:33:56
29108443   1970-01-07 23:22:04
29109993   1970-01-08 15:03:14
29111539   1970-01-04 00:53:41
29112154   1970-01-08 23:01:07
Name: time, Length: 17710, dtype: datetime64[ns]
--------------------------------------------------
DatetimeIndex(['1970-01-01 18:09:40', '1970-01-10 02:11:10',
               '1970-01-05 15:08:02', '1970-01-06 23:03:03',
               '1970-01-09 11:26:50', '1970-01-02 16:25:07',
               '1970-01-04 15:52:57', '1970-01-01 10:13:36',
               '1970-01-09 15:26:06', '1970-01-08 23:52:02',
               ...
               '1970-01-07 10:03:36', '1970-01-09 11:44:34',
               '1970-01-04 08:07:44', '1970-01-04 15:47:47',
               '1970-01-08 01:24:11', '1970-01-01 10:33:56',
               '1970-01-07 23:22:04', '1970-01-08 15

In [6]:
#3  构造一些特征，执行的警告是因为我们的操作是复制，loc是直接放入
print(type(data))
print('-' * 50)
# data['day'] = time_value.day
# data['hour'] = time_value.hour
# data['weekday'] = time_value.weekday
#日期，是否是周末，小时对于个人行为的影响是较大的,所以才做下面的处理
#data.shape[1]是代表插入到最后的意思
data.insert(data.shape[1], 'day', time_value.day)
data.insert(data.shape[1], 'hour', time_value.hour)
data.insert(data.shape[1], 'weekday', time_value.weekday)

print('-' * 50)
print('before')
#row_id  x  y  accuracy  time place_id  day  hour  weekday
print(data)
# 把时间戳特征删除
data = data.drop(['time'], axis=1)#axis=1代表列
print('-' * 50)
print('after')
#row_id    x   y  accuracy  place_id  day  hour  weekday
print(data)

<class 'pandas.core.frame.DataFrame'>
--------------------------------------------------
--------------------------------------------------
before
            row_id       x       y  accuracy    time    place_id  day  hour  \
600            600  1.2214  2.7023        17   65380  6683426742    1    18   
957            957  1.1832  2.6891        58  785470  6683426742   10     2   
4345          4345  1.1935  2.6550        11  400082  6889790653    5    15   
4735          4735  1.1452  2.6074        49  514983  6822359752    6    23   
5580          5580  1.0089  2.7287        19  732410  1527921905    9    11   
...            ...     ...     ...       ...     ...         ...  ...   ...   
29100203  29100203  1.0129  2.6775        12   38036  3312463746    1    10   
29108443  29108443  1.1474  2.6840        36  602524  3533177779    7    23   
29109993  29109993  1.0240  2.7238        62  658994  6424972551    8    15   
29111539  29111539  1.2032  2.6796        87  262421  353317777

In [7]:
#星期天，实际weekday的值是6
per = pd.Period('2023-07-13 22:00', 'H')
per.weekday

3

In [8]:
#观察数据，看下是否有异常值
data.describe()

,row_id,x,y,accuracy,place_id,day,hour,weekday
count,1.771000e+04,17710.000000,17710.000000,17710.000000,1.771000e+04,17710.000000,17710.000000,17710.000000
mean,1.450569e+07,1.122538,2.632309,82.482101,5.129895e+09,5.101863,11.485545,3.092377
std,8.353805e+06,0.077086,0.070144,113.613227,2.357399e+09,2.709287,6.932195,1.680218
min,6.000000e+02,1.000100,2.500100,1.000000,1.012024e+09,1.000000,0.000000,0.000000
25%,7.327816e+06,1.049200,2.573800,25.000000,3.312464e+09,3.000000,6.000000,2.000000
50%,1.443071e+07,1.123300,2.642300,62.000000,5.261906e+09,5.000000,12.000000,3.000000
75%,2.163463e+07,1.190500,2.687800,75.000000,6.766325e+09,7.000000,17.000000,4.000000
max,2.911215e+07,1.249900,2.749900,1004.000000,9.980711e+09,10.000000,23.000000,6.000000


In [9]:
# # 为什么，把签到数量少于n个目标位置删除，place_id是标签，即目标值
#？？？？？？？？？？？没看懂这行代码的意义
# 答案
# 这段代码的目的是对数据集中的place_id进行分组，
# 并统计每个place_id出现的次数。让我为您解释一下这么做的原因和意义。
#
# 在这段代码中，data代表数据集，place_id是数据中的一个属性，表示地点或位置的唯一标识符。
# 通过对place_id进行分组，然后计算每个分组中的样本数量，可以获得每个地点被记录的次数。
# 这在某些情况下可能具有重要的分析意义，下面是一些可能的原因和意义：
#
# 地点热度分析： 通过统计每个地点的出现次数，您可以了解哪些地点在数据集中出现得更频繁，从而推断出这些地点可能是热门的或受欢迎的地点。
# 这对于了解用户的兴趣、行为和偏好可能非常有用。
#
# 异常检测： 统计地点的出现次数还可以帮助识别可能的异常或离群点。
# 如果某个地点的出现次数远远高于其他地点，那么可能存在一些异常情况，需要进一步调查。
#
# 数据质量分析： 统计每个地点的出现次数可以帮助您了解数据集的分布情况。
# 如果某些地点的出现次数极少，可能表示数据集中的这些地点信息缺失或不准确。
#
# 推荐系统： 对地点的出现次数进行分析还可以用于构建推荐系统。
# 根据用户的历史访问地点以及地点的热度，可以为用户推荐可能感兴趣的地点。
#
# 总之，对place_id进行分组并统计出现次数可以提供关于地点在数据集中的分布、热度和重要性等信息，这些信息可以在分析、预测和应用开发中发挥重要作用。

place_count = data.groupby('place_id').count() #对place_id进行分组，然后统计每个place_id的个数
print(place_count)
print('-' * 50)
place_count['x'].describe()# 对x进行统计 count是个数，mean是平均值，std是标准差，min是最小值，max是最大值
# # 把index变为0,1,2，3,4,5,6这种效果，从零开始排，原来的index是row_id
#只选择去的人大于3的数据，认为1,2,3的是噪音，这个地方去的人很少，不用推荐给其他人
tf = place_count[place_count.row_id > 3].reset_index()
print(tf)  #剩余的签到地点 805-239=566个地点, 566个地点是噪音被删除了
print('-' * 50)

            row_id     x     y  accuracy   day  hour  weekday
place_id                                                     
1012023972       1     1     1         1     1     1        1
1057182134       1     1     1         1     1     1        1
1059958036       3     3     3         3     3     3        3
1085266789       1     1     1         1     1     1        1
1097200869    1044  1044  1044      1044  1044  1044     1044
...            ...   ...   ...       ...   ...   ...      ...
9904182060       1     1     1         1     1     1        1
9915093501       1     1     1         1     1     1        1
9946198589       1     1     1         1     1     1        1
9950190890       1     1     1         1     1     1        1
9980711012       5     5     5         5     5     5        5

[805 rows x 7 columns]
--------------------------------------------------
       place_id  row_id     x     y  accuracy   day  hour  weekday
0    1097200869    1044  1044  1044      1044  1044 

In [10]:
# 根据设定的地点目标值，对原本的样本进行过滤
#isin可以过滤某一列要在一组值
data = data[data['place_id'].isin(tf.place_id)]
data.shape

(16918, 8)

In [11]:
#  取出数据当中的特征值和目标值
y = data['place_id'] #目标值
# 删除目标值，保留特征值，
x = data.drop(['place_id'], axis=1)
# 删除无用的特征值
x = x.drop(['row_id'], axis=1)
print(x.shape)
print(x.columns)

#代码解释：
# 这段代码的目的是将数据集中的特征值和目标值分开，然后对特征值进行预处理。
#
# 让我解释一下每一步的具体目的和用途：
#
# y = data['place_id']：这一步将目标值提取出来，place_id列中的值将被存储在变量y中。
# 目标值通常是我们希望预测或分类的内容。
#
# x = data.drop(['place_id'], axis=1)：
# 这一步删除了数据集中的place_id列，即将目标值从特征值中分离出来，得到一个新的DataFrame x，
# 其中包含所有其他特征。
#
# x = x.drop(['row_id'], axis=1)：这一步删除了row_id列，
# 如果row_id列不包含有关目标值的信息，那么它可以被视为无用的特征，因此被删除。
#
# print(x.shape)：打印输出特征值数据的形状，即数据集中的样本数量和特征数量。
#
# print(x.columns)：打印输出特征值数据的列名，即所有特征的名称。
#
# 综上所述，这段代码通过将目标值和无关特征从数据集中分离出来，
# 得到一个只包含有效特征的数据集。这是在进行数据预处理和特征工程时的常见步骤。
# 分离出目标值和无关特征可以使特征处理更加简洁和准确，从而有助于构建和训练机器学习模型。

(16918, 6)
Index(['x', 'y', 'accuracy', 'day', 'hour', 'weekday'], dtype='object')


In [12]:
# 数据的分割，特征工程结束了，开始机器训练，构建模型
#//////////////////////////////////////////////
#//////////////////////////////////////////////
#//////////////////////////////////////////////
#//////////////////////////////////////////////

In [15]:
from sklearn.model_selection import train_test_split # 数据的分割
from sklearn.preprocessing import StandardScaler # 数据的标准化
from sklearn.neighbors import KNeighborsClassifier

In [16]:
# 进行数据的分割训练集合测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

# 特征工程（标准化）,下面3行注释，一开始我们不进行标准化，看下效果，目标值要不要标准化？
std = StandardScaler()
# #
# # # 对测试集和训练集的特征值进行标准化,服务于knn fit
x_train = std.fit_transform(x_train)
# # transform返回的是copy，不在原有的输入对象中去修改
# print(id(x_test))
print(std.mean_)
print(std.var_)
x_test = std.transform(x_test)  #transfrom不再进行均值和方差的计算，是在原有的基础上去标准化
print('-' * 50)
# print(id(x_test))
print(std.mean_)
print(std.var_)

# # 进行算法流程 # 超参数，可以通过设置n_neighbors=5，来调整结果好坏
knn = KNeighborsClassifier(n_neighbors=6)

# # fit， predict,score，训练
knn.fit(x_train, y_train)
# # #
# # # 得出预测结果
y_predict = knn.predict(x_test)
# #
print("预测的目标签到位置为：", y_predict)
# # #
# # # # 得出准确率
print("预测的准确率:", knn.score(x_test, y_test))

[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]
--------------------------------------------------
[ 1.12295735  2.63237278 81.34938525  5.10064628 11.44293821  3.10135561]
[5.98489138e-03 4.86857391e-03 1.19597480e+04 7.32837915e+00
 4.83742660e+01 2.81838404e+00]
预测的目标签到位置为： [5689129232 1097200869 2355236719 ... 4932578245 6424972551 5095999304]
预测的准确率: 0.484160756501182


In [17]:
print(max(time_value))

1970-01-10 02:23:38


In [1]:
from sklearn.model_selection import GridSearchCV

In [2]:
#网格搜索时讲解
# # 构造一些参数的值进行搜索
# dict---<string,list(int)/list(string)>
param = {"n_neighbors": [3, 5, 10, 12, 15],'weights':['uniform', 'distance']}
# print(type(param))
# print(type(param["n_neighbors"]))

In [3]:
# 进行网格搜索，cv=3是3折交叉验证，用其中2折训练，1折验证
gc = GridSearchCV(knn, param_grid=param, cv=3)

gc.fit(x_train, y_train)  #你给它的x_train，它又分为训练集，验证集

# 预测准确率，为了给大家看看
print("在测试集上准确率：", gc.score(x_test, y_test))

print("在交叉验证当中最好的结果：", gc.best_score_)

print("选择最好的模型是：", gc.best_estimator_)

print("每个超参数每次交叉验证的结果：", gc.cv_results_)
gc.cv_results_

NameError: name 'knn' is not defined